# Define the sinusoid signal and plot it.

#### Zu-Cheng Chen, Feb 2019

In [49]:
# load packages
using PyPlot, WAV
using Plots
using FFTW
using DSP
import LinearAlgebra:norm

In [50]:
?firfilt

search: firfilt FIRFilter FilterCoefficients filter_type_differentiator



No documentation found.

`DSP.firfilt` is a `Function`.

```
# 1 method for generic function "firfilt":
[1] firfilt(h, x) in DSP at deprecated.jl:53
```


## Define the sinusoid signal.
The signal is  by
$$
s(t) = A \sin(2\pi \phi(t)),
$$
where $A$ is the $snr$, and the phase is given by
$$
\phi(t) = a_1 t + a_2 t^2 + a_3 t^3.
$$

In [14]:
"""
Generate a quadratic chirp signal
S = CRCBGENQSIG(X,SNR,C)
Generates a quadratic chirp signal S. X is the vector of
time stamps at which the samples of the signal are to be computed. SNR is
the matched filtering signal-to-noise ratio of S and C is the vector of
three coefficients [a1, a2, a3] that parametrize the phase of the signal:
a1*t+a2*t^2+a3*t^3. 
"""
function crcbgenqcsig(dataX, snr::Real, qcCoefs::Array)
    phaseVec = qcCoefs[1]*dataX + qcCoefs[2]*dataX.^2 + qcCoefs[3]*dataX.^3;
    sigVec = sin.(2π*phaseVec)
    sigVec = snr*sigVec/norm(sigVec)
end

# test the function
crcbgenqcsig(1, 1, [1, 1, 1]), crcbgenqcsig([1,2], 1, [1, 1, 1])

(-1.0, [-0.209529, -0.977802])

In [13]:
?crcbgenqcsig

search: crcbgenqcsig



SinSig(dataT, snr, f0, f1, ϕ0) generates a sinusoid signal S.  dataT is the vector of time stamps at which the samples  of the signal are to be computed.  snr is the matched filtering signal-to-noise ratio of S,  and [f0, ϕ0] are coefficients that parametrize the phase of the signal:  2π*f0*dataT + ϕ0.

---

SinSig(dataT, snr, f0, f1, ϕ0) generates a sinusoid signal S.  dataT is the vector of time stamps at which the samples  of the signal are to be computed.  snr is the matched filtering signal-to-noise ratio of S,  and [f0, ϕ0] are coefficients that parametrize the phase of the signal:  2π*f0*dataT + ϕ0.


## Spectrogram demo

In [18]:
sampFreq = 1024;
nSamples = 2048;
timeVec = (0:(nSamples-1))/sampFreq;
timeVec = collect(timeVec);

In [20]:
# Quadratic chirp signal
# Signal parameters
a1=10;
a2=3;
a3=10;
A = 10;

In [21]:
# Generate signal
sigVec = crcbgenqcsig(timeVec,A,[a1,a2,a3]);

In [33]:
# Make spectrogram with different time-frequency resolution
S = spectrogram(sigVec, 128, 127; fs=sampFreq)

DSP.Periodograms.Spectrogram{Float64,Frequencies}([0.000344835 0.000374765 … 7.29438e-7 7.50574e-8; 0.00477264 0.00490573 … 1.47953e-6 1.62934e-7; … ; 3.42716e-7 2.81863e-7 … 5.94165e-7 6.80877e-7; 1.71254e-7 1.40846e-7 … 2.96819e-7 3.40153e-7], [0.0, 8.0, 16.0, 24.0, 32.0, 40.0, 48.0, 56.0, 64.0, 72.0  …  440.0, 448.0, 456.0, 464.0, 472.0, 480.0, 488.0, 496.0, 504.0, 512.0], 0.0625:0.0009765625:1.9375)

In [38]:
t = time(S);
f = freq(S);

In [48]:
flipud(log10.(power(S)))

UndefVarError: UndefVarError: flipud not defined

In [44]:
imshow(flipud(log10.(power(S))), extent=[first(t), last(t), fs*first(f), fs*last(f)], aspect="auto")

UndefVarError: UndefVarError: flipud not defined

In [ ]:
fi

In [ ]:
figure;
spectrogram(sigVec, 256,250,[],sampFreq);

%%
% Make plots independently of the spectrogram function
[S,F,T]=spectrogram(sigVec, 256,250,[],sampFreq);
figure;
imagesc(T,F,abs(S));axis xy;
xlabel('Time (sec)');
ylabel('Frequency (Hz)');